In [1]:
import torch
import abmap

In [6]:
# Load AbMAP 
# Using Bepler-Berger as foundational model
abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_beplerberger_H_epoch50.pt', plm_name='beplerberger')
abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_beplerberger_L_epoch50.pt', plm_name='beplerberger')

# Using ESM1b
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm1b_H.pt', plm_name='esm1b')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm1b_L.pt', plm_name='esm1b')

# Using ProtBert
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_protbert_H.pt', plm_name='protbert')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_protbert_L.pt', plm_name='protbert')

beplerberger loaded to cuda:0
Loaded the Pre-trained Model!
beplerberger loaded to cuda:0
Loaded the Pre-trained Model!


In [8]:
# Get embedding for one sequence
demo_seq = 'EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS'

# Contrastive augmentation (PLM, mutagenesis, CDR focus)
x = abmap.ProteinEmbedding(demo_seq, chain_type='H', embed_device=torch.device('cpu'))
x.create_cdr_specific_embedding(embed_type='beplerberger', k=50)

# Pass the augmented embedding to AbMAP to get final embedding
with torch.no_grad():
    embed_var = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='variable') # residue-level embeddings
    embed_fl = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='fixed') # fixed-length

UnboundLocalError: local variable 'Uniprot21' referenced before assignment

In [18]:
embed_var.shape, embed_fl.shape

(torch.Size([1, 31, 256]), torch.Size([1, 512]))

In [ ]:
# Get embeddings for a fasta file of sequences